In [1]:
pip install simpy networkx

In [2]:
import simpy
import networkx as nx

# Define a frame class to represent the frames being transferred in the network
class Frame:
    def __init__(self, env, source, destination, frame_id):
        self.env = env  # Simulation environment
        self.source = source  # Source node of the frame
        self.destination = destination  # Destination node of the frame
        self.creation_time = env.now  # Timestamp of frame creation
        self.status = 'in transit'  # Status of the frame: 'in transit', 'arrived', 'discarded'
        self.path_traveled = [source]  # Path the frame has traveled
        self.frame_id = frame_id  # Unique identifier for the frame

        self.storage_time_limit = 10  # Maximum allowed storage time
        self.time_in_storage = 0  # Time spent in storage
        self.temporal_frame_length = 5  # Length of the frame in time units
        self.guard_time = 3  # Guard time for frame integrity

    def update_storage_time(self, time_spent):
        self.time_in_storage += time_spent
        if self.time_in_storage > self.storage_time_limit:
            self.status = 'discarded'

# Define a network node (e.g., user, router, receiver)
class NetworkNode:
    def __init__(self, env, name, node_type, k=1):
        self.env = env  # Simulation environment
        self.name = name  # Name of the node
        self.node_type = node_type  # Type of node: 'sender', 'router', or 'receiver'
        self.queue = simpy.Store(env)  # Queue to store frames
        self.resource = simpy.Resource(env, capacity=k)  # Resource to handle k simultaneous processes

    def process_frames(self):
        while True:
            frame = yield self.queue.get()  # Get the next frame from the queue
            with self.resource.request() as request:
                yield request  # Wait until the resource is available
                if self.node_type in ['router', 'receiver']:
                    # Simulate header processing delay for routers and receivers
                    processing_delay = 1  # Example processing delay
                    yield self.env.timeout(processing_delay)
                    frame.update_storage_time(processing_delay)
                    if frame.status == 'discarded':
                        print(f"Time {self.env.now}: Frame {frame.frame_id} discarded at {self.name}")
                        continue  # Skip further processing if frame is discarded
                    #print(f"Time {self.env.now}: {self.name} processed Frame {frame.frame_id} ")
                if self.name != frame.destination:
                    # If the current node is not the destination, forward the frame
                    self.env.process(route_frame(self.env, frame, network, self))
                else:
                    # If the current node is the destination, deliver the frame
                    frame.status = 'arrived'
                    print(f"Time {self.env.now}: Frame {frame.frame_id} delivered at {self.name}, from {frame.source} to {frame.destination}")

# Define a function to generate frames at a given source node
def generate_frames(env, source_node, destination, interval, max_packets):
    packet_count = 0  # Counter for the number of packets generated
    while packet_count < max_packets:
        yield env.timeout(interval)  # Wait for the next frame generation interval
        # Create a new frame with example attributes
        frame = Frame(env, source_node.name, destination, packet_count + 1)
        source_node.queue.put(frame)  # Put the frame in the source node's queue
        print(f"\nTime {env.now}: Frame {frame.frame_id} created at {source_node.name}, destined for {destination}")
        frames.append(frame)  # Collect the frame for visualization
        packet_count += 1  # Increment the packet counter

# Define a function to route frames from one node to the next hop
def route_frame(env, frame, network, current_node):
    # Calculate the shortest path from the source to the destination
    path = nx.shortest_path(network, source=frame.source, target=frame.destination, weight='weight')
    # Determine the next hop in the path
    next_hop = path[path.index(current_node.name) + 1]
    # Get the next node object from the dictionary
    next_node = node_dict[next_hop]
    # Simulate the transmission delay based on the edge weight
    transmission_delay = network[current_node.name][next_hop]['weight']
    yield env.timeout(transmission_delay)
    frame.update_storage_time(transmission_delay)
    if frame.status == 'discarded':
        print(f"Time {env.now}: Frame {frame.frame_id} discarded en route at {current_node.name}")
        return  # Stop forwarding if frame is discarded
    frame.path_traveled.append(next_hop)  # Record the path traveled
    print(f"Time {env.now}: Frame {frame.frame_id} forwarded from {current_node.name} to {next_hop}")
    # Put the frame in the next node's queue
    next_node.queue.put(frame)

# Create a directed graph to represent the network topology
network = nx.DiGraph()

# Function to create a 3x3 grid of nodes and edges
def create_grid_graph(network, rows, cols):
    nodes = []
    for i in range(rows):
        for j in range(cols):
            node = f'Node_{i}_{j}'
            nodes.append(node)
            network.add_node(node)
            # Connect to the right neighbor
            if j < cols - 1:
                network.add_edge(node, f'Node_{i}_{j+1}', weight=1)
                network.add_edge(f'Node_{i}_{j+1}', node, weight=1)
            # Connect to the bottom neighbor
            if i < rows - 1:
                network.add_edge(node, f'Node_{i+1}_{j}', weight=1)
                network.add_edge(f'Node_{i+1}_{j}', node, weight=1)
    return nodes

# Create a 3x3 grid of nodes
grid_size = 3
nodes = create_grid_graph(network, grid_size, grid_size)

# Create the simulation environment
env = simpy.Environment()

# Define node types (for simplicity, all nodes are routers except the first and last)
node_types = {}
for i in range(grid_size):
    for j in range(grid_size):
        if i == 0 and j == 0:
            node_types[f'Node_{i}_{j}'] = 'sender'
        elif i == grid_size - 1 and j == grid_size - 1:
            node_types[f'Node_{i}_{j}'] = 'receiver'
        else:
            node_types[f'Node_{i}_{j}'] = 'router'

# Create network node objects and store them in a dictionary
node_dict = {name: NetworkNode(env, name, node_types[name], k=2) for name in nodes}

# List to store all frames for visualization
frames = []

# Define the maximum number of packets to generate
max_packets = 10

# Start the frame generation process at the sender node, creating frames destined for the receiver node
env.process(generate_frames(env, node_dict['Node_0_0'], 'Node_2_2', interval=10, max_packets=max_packets))

# Start the frame processing process for each node in the network
for node in node_dict.values():
    env.process(node.process_frames())

# Run the simulation until all packets are generated and processed
while len(frames) < max_packets or any(frame.status == 'in transit' for frame in frames):
    env.step()

print(f"\nSimulation completed after generating and processing {max_packets} packets.")



Time 10: Frame 1 created at Node_0_0, destined for Node_2_2
Time 11: Frame 1 forwarded from Node_0_0 to Node_0_1
Time 13: Frame 1 forwarded from Node_0_1 to Node_0_2
Time 15: Frame 1 forwarded from Node_0_2 to Node_1_2
Time 17: Frame 1 forwarded from Node_1_2 to Node_2_2
Time 18: Frame 1 delivered at Node_2_2, from Node_0_0 to Node_2_2

Time 20: Frame 2 created at Node_0_0, destined for Node_2_2
Time 21: Frame 2 forwarded from Node_0_0 to Node_0_1
Time 23: Frame 2 forwarded from Node_0_1 to Node_0_2
Time 25: Frame 2 forwarded from Node_0_2 to Node_1_2
Time 27: Frame 2 forwarded from Node_1_2 to Node_2_2
Time 28: Frame 2 delivered at Node_2_2, from Node_0_0 to Node_2_2

Time 30: Frame 3 created at Node_0_0, destined for Node_2_2
Time 31: Frame 3 forwarded from Node_0_0 to Node_0_1
Time 33: Frame 3 forwarded from Node_0_1 to Node_0_2
Time 35: Frame 3 forwarded from Node_0_2 to Node_1_2
Time 37: Frame 3 forwarded from Node_1_2 to Node_2_2
Time 38: Frame 3 delivered at Node_2_2, from Node